# 1. WebScraping

### Importando bibliotecas

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

### Criando variáveis

In [3]:
lista_cidades = {}

estados = ['sp', 'mg']

dic = {
    " ": "_",
    "'": "",
    "â": "a",
    "ã": "a",
    "á": "a",
    "é": "e",
    "ê": "e",
    "í": "i",
    "ó": "o",
    "õ": "o",
    "ô": "o",
    "ú": "u",
    "ç": "c"
    
}

In [4]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [5]:
for estado in estados: 
    headers = {'user-agent': 'Mozilla/5.0'}
    url = f'https://www.feriados.com.br/feriados-estado-{estado}.php?ano=2023'
    response  = requests.get(url, headers = headers)
    response_text = response.text
    
    soup = BeautifulSoup(response_text, 'lxml')
    
    cidades = soup.find('div', id='div_outras_cidades').find_all('a')
    for c in cidades:
        cidade = c.text
        cidade_padronizado = replace_all(cidade.lower(), dic)
        lista_cidades[f'{cidade} - {estado.upper()}'] = f'{cidade_padronizado}-{estado}'

In [6]:
lista_completa = []

for nome, link in lista_cidades.items():
    
    lista_estado = []
    lista_cidade = []
    lista_datas = []
    lista_feriados = []

    
    url = f'https://www.feriados.com.br/feriados-{link}.php?ano=2023'

    
    response = requests.get(url)
    response_text = response.text

    soup = BeautifulSoup(response_text, 'lxml')

    feriads = soup.find('ul', class_='multi-column')
    feriados = feriads.find_all('li')


    for feriado in feriados:
        lista_cidade.append('-'.join(nome.split('-')[:-1]).strip())
        lista_estado.append(nome.split('-')[-1].strip())
        
        lista = feriado.text.split('-')
        lista_datas.append(lista[0].strip())
        lista_feriados.append((' '.join(lista[1:])).strip().title())

    lista_completa.extend(list(zip(lista_cidade,lista_estado,lista_datas,lista_feriados)))

In [7]:
lista_completa[0:5]

[('Adamantina', 'SP', '01/01/2023', 'Ano Novo'),
 ('Adamantina', 'SP', '20/02/2023', 'Carnaval'),
 ('Adamantina', 'SP', '21/02/2023', 'Carnaval'),
 ('Adamantina', 'SP', '22/02/2023', 'Carnaval'),
 ('Adamantina', 'SP', '07/04/2023', 'Sexta Feira Santa')]

# 2. Criação do DataFrame e tratamento

In [8]:
columns = ['Cidade','Estado','Data','Feriado']

In [9]:
df = pd.DataFrame(lista_completa, columns = columns)

In [10]:
df.head(10)

,Cidade,Estado,Data,Feriado
0,Adamantina,SP,01/01/2023,Ano Novo
1,Adamantina,SP,20/02/2023,Carnaval
2,Adamantina,SP,21/02/2023,Carnaval
3,Adamantina,SP,22/02/2023,Carnaval
4,Adamantina,SP,07/04/2023,Sexta Feira Santa
5,Adamantina,SP,21/04/2023,Dia De Tiradentes
6,Adamantina,SP,01/05/2023,Dia Do Trabalho
7,Adamantina,SP,08/06/2023,Corpus Christi
8,Adamantina,SP,13/06/2023,Dia De Santo Antônio
9,Adamantina,SP,09/07/2023,Revolução Constitucionalista


In [11]:
df.Feriado.value_counts()[:10]

Carnaval                    4497
Feriado Municipal           1976
Corpus Christi              1598
Sexta Feira Santa           1549
Ano Novo                    1498
Dia Do Professor            1498
Natal                       1498
Proclamação Da República    1498
Dia De Finados              1498
Dia Do Servidor Público     1498
Name: Feriado, dtype: int64

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.drop(index = df[df['Feriado'] == 'Finados'].index, axis = 0, inplace=True)
df.drop(index = df[df['Feriado'] == 'Sexta Feira Da Paixão'].index, axis = 0, inplace=True)
df.drop(index = df[df['Feriado'] == 'Terça De Carnaval'].index, axis = 0, inplace=True)
df.replace('Dia Da Consciência Negra   Lei No. 3.249, De 18 De Novembro De 2008','Dia Da Consciência Negra', inplace=True)
df.replace('Dia Da Consciência Negra   Lei No 1.914 De 04 De Dezembro De 2007','Dia Da Consciência Negra', inplace=True)
df.replace('Dia Da Consciência Negra   Lei 4.731 22/12/2008.','Dia Da Consciência Negra', inplace=True)

In [14]:
df = df.reset_index(drop=True)

In [15]:
df.Feriado.value_counts()

Carnaval                                                                      4494
Feriado Municipal                                                             1976
Ano Novo                                                                      1498
Nossa Senhora Aparecida                                                       1498
Natal                                                                         1498
                                                                              ... 
Sagrado Coração De Jesus                                                         1
Fundação Da Cidade De Palmital E Dia De São Sebastião, Padroeiro Da Cidade       1
Emancipação De Osasco                                                            1
São João Batista                                                                 1
Santa Rita                                                                       1
Name: Feriado, Length: 130, dtype: int64

In [16]:
feriados_nacionais = ['Carnaval','Ano Novo','Nossa Senhora Aparecida','Natal','Proclamação Da República',
                      'Dia De Finados','Dia Do Servidor Público','Dia Do Professor','Independência Do Brasil',
                     'Corpus Christi','Dia Do Trabalho','Dia De Tiradentes','Sexta Feira Santa']

conditions = [
    (df['Feriado'].isin(feriados_nacionais)),
    (df['Feriado'] == 'Revolução Constitucionalista')
     ]

labels = ['Feriado Nacional', 'Feriado Estadual']

In [17]:
df['Tipo'] = np.select(conditions, labels, default = 'Feriado Municipal')

In [18]:
df.head()

,Cidade,Estado,Data,Feriado,Tipo
0,Adamantina,SP,01/01/2023,Ano Novo,Feriado Nacional
1,Adamantina,SP,20/02/2023,Carnaval,Feriado Nacional
2,Adamantina,SP,21/02/2023,Carnaval,Feriado Nacional
3,Adamantina,SP,22/02/2023,Carnaval,Feriado Nacional
4,Adamantina,SP,07/04/2023,Sexta Feira Santa,Feriado Nacional


In [19]:
df.to_csv('lista_feriados.csv', sep=';', encoding='UTF-8', index=False)